In [69]:
import openmeteo_requests
import requests_cache
import pandas as pd
import numpy as np
from retry_requests import retry
import requests
import datetime as dt

In [70]:
fl = pd.read_csv('AutomatedPitchTagging/AutoTaggedCSVs/2023 Frontier League Autotagged.csv')

In [71]:
fl.Date = pd.to_datetime(fl['Date'], errors='coerce').dt.strftime('%Y-%m-%d')

In [72]:
[c for c in fl.columns]

['Unnamed: 0',
 'PitchNo',
 'Date',
 'Time',
 'PAofInning',
 'PitchofPA',
 'Pitcher',
 'PitcherId',
 'PitcherThrows',
 'PitcherTeam',
 'Batter',
 'BatterId',
 'BatterSide',
 'BatterTeam',
 'PitcherSet',
 'Inning',
 'Top/Bottom',
 'Outs',
 'Balls',
 'Strikes',
 'TaggedPitchType',
 'AutoPitchType',
 'PitchCall',
 'KorBB',
 'HitType',
 'PlayResult',
 'OutsOnPlay',
 'RunsScored',
 'Notes',
 'RelSpeed',
 'VertRelAngle',
 'HorzRelAngle',
 'SpinRate',
 'SpinAxis',
 'Tilt',
 'RelHeight',
 'RelSide',
 'Extension',
 'VertBreak',
 'InducedVertBreak',
 'HorzBreak',
 'PlateLocHeight',
 'PlateLocSide',
 'ZoneSpeed',
 'VertApprAngle',
 'HorzApprAngle',
 'ZoneTime',
 'ExitSpeed',
 'Angle',
 'Direction',
 'HitSpinRate',
 'PositionAt110X',
 'PositionAt110Y',
 'PositionAt110Z',
 'Distance',
 'LastTrackedDistance',
 'Bearing',
 'HangTime',
 'pfxx',
 'pfxz',
 'x0',
 'y0',
 'z0',
 'vx0',
 'vy0',
 'vz0',
 'ax0',
 'ay0',
 'az0',
 'HomeTeam',
 'AwayTeam',
 'Stadium',
 'Level',
 'League',
 'GameID',
 'PitchUUID

In [73]:
team_mapping = {
    'Trois Rivieres Aigles ': 'Trois Rivieres Aigles',
    'Trois Rivieres Aigles': 'Trois Rivieres Aigles',
    'Evansville Otters': 'Evansville Otters',
    'Florence Yalls': 'Florence Yalls',
    'Gateway Grizzlies': 'Gateway Grizzlies',
    'Joliet Slammers': 'Joliet Slammers',
    'Lake Erie Crushers': 'Lake Erie Crushers',
    'New Jersey Jackals': 'New Jersey Jackals',
    'New York Boulders': 'New York Boulders',
    'Ottawa Titans': 'Ottawa Titans',
    'Quebec Capitales': 'Quebec Capitales',
    'Schaumburg Boomers': 'Schaumburg Boomers',
    'Sussex County Miners': 'Sussex County Miners',
    'Tri-City ValleyCats': 'Tri-City ValleyCats',
    'Tri-City Valleycats': 'Tri-City ValleyCats',
    'ValleyCats': 'Tri-City ValleyCats',  
    'Washington Wild Things': 'Washington Wild Things',
    'Windy City Thunderbolts': 'Windy City Thunderbolts',
    'Tri-City-Valley-Cats': 'Tri-City ValleyCats',
    'TC Valley-Cats': 'Tri-City ValleyCats',
    'TCVC': 'Tri-City ValleyCats',
    'Tri city valleycats 2023': 'Tri-City ValleyCats',
    'Empire State Greys ': 'Empire State Greys',
    'Empire State Greys': 'Empire State Greys',
    'Les Aigles De Trois Rivieres 2023': 'Trois Rivieres Aigles',
    "Florence Y'alls": 'Florence Yalls',
    'Boulders New york': 'New York Boulders',
    'Trois-Rivieres Aigles': 'Trois Rivieres Aigles',
    'Windy City ThunderBolts': 'Windy City Thunderbolts'
}


fl['HomeTeam'] = fl['HomeTeam'].map(team_mapping)
fl['AwayTeam'] = fl['AwayTeam'].map(team_mapping)

In [74]:
pd.Series(team_mapping.values()).unique()

array(['Trois Rivieres Aigles', 'Evansville Otters', 'Florence Yalls',
       'Gateway Grizzlies', 'Joliet Slammers', 'Lake Erie Crushers',
       'New Jersey Jackals', 'New York Boulders', 'Ottawa Titans',
       'Quebec Capitales', 'Schaumburg Boomers', 'Sussex County Miners',
       'Tri-City ValleyCats', 'Washington Wild Things',
       'Windy City Thunderbolts', 'Empire State Greys'], dtype=object)

In [75]:
coords_map = {
    'Trois Rivieres Aigles': (46.34, 72.54),
    'Evansville Otters': (37.97, 87.57),
    'Florence Yalls': (38.99, 84.64),
    'Gateway Grizzlies': (38.33, 90.08),
    'Joliet Slammers': (41.52, 88.08),
    'Lake Erie Crushers': (41.45, 82.03),
    'New Jersey Jackals': (39.93, 74.87),
    'New York Boulders': (41.10, 74.02),
    'Ottawa Titans': (45.42, 75.69),
    'Quebec Capitales': (46.81, 71.20),
    'Schaumburg Boomers': (42.03, 88.08),
    'Sussex County Miners': (41.1288, 74.68),
    'Tri-City ValleyCats': (42.72, 73.69),
    'Washington Wild Things': (40.1740, 80.24),
    'Windy City Thunderbolts': (41.66, 87.75),
    'Empire State Greys': (44.27, 73.97),
}

fl['coord_lat'] = fl['HomeTeam'].map({team: coords[0] for team, coords in coords_map.items()})
fl['coord_long'] = fl['HomeTeam'].map({team: coords[1] for team, coords in coords_map.items()})

In [76]:
single_day = fl[['coord_lat', 'coord_long', 'Date']].drop_duplicates()

In [77]:
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
df_with_weather = pd.DataFrame(columns=["date", "temperature_2m", "relative_humidity_2m", "surface_pressure", "elevation", "latitude", "longitude"])

for index, row in single_day.iterrows():
    latitude = row['coord_lat']
    longitude = row['coord_long']
    date = row['Date']

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": date,
        "end_date": date,
        "hourly": ["temperature_2m", "relative_humidity_2m", "pressure_msl"]
    }
    responses = openmeteo.weather_api(url, params=params)

    response = responses[0]

    hourly = response.Hourly()
    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s"),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s"),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
        "relative_humidity_2m": hourly.Variables(1).ValuesAsNumpy(),
        "pressure_msl": hourly.Variables(2).ValuesAsNumpy(),
        "elevation": response.Elevation(),
        "latitude": latitude,
        "longitude": longitude,
    }

    hourly_dataframe = pd.DataFrame(data=hourly_data)

    median_data = {
        "date": [date],
        "temperature_2m": [np.percentile(hourly_dataframe["temperature_2m"], 75)],
        "relative_humidity_2m": [np.percentile(hourly_dataframe["relative_humidity_2m"], 75)],
        "surface_pressure": [np.percentile(hourly_dataframe["pressure_msl"], 75)],
        "elevation": [hourly_dataframe["elevation"].iloc[0]],
        "latitude": [latitude],
        "longitude": [longitude],
    }

    median_dataframe = pd.DataFrame(data=median_data)
    df_with_weather = pd.concat([df_with_weather, median_dataframe], ignore_index=True)

In [78]:
fl = pd.merge(df_with_weather, fl, left_on=['date', 'latitude', 'longitude'], right_on=['Date', 'coord_lat', 'coord_long'], how='right')

In [79]:
fl = fl.dropna(subset=['temperature_2m', 'relative_humidity_2m', 'elevation', 'surface_pressure', 'elevation', 
                       'RelSpeed', 'HorzRelAngle', 'VertRelAngle', 'Extension', 'RelSide', 'RelHeight', 'SpinRate', 
                       'Tilt', 'SpinAxis', 'yt_SeamLat', 'yt_SeamLong'])

In [80]:
def calculate_air_density(temperature, elevation):
    temperature += 273.15
    p0 = 101325
    T0 = 288.15
    g = 9.81
    L = 0.0065
    R = 8.314
    M = 0.030
    T = T0 - L * elevation
    p = p0 * (1 - (L * elevation) / (T0)) ** ((g * M) / (R * L))
    density = (p * M) / (R * temperature)
    return density

fl['air_density'] = fl.apply(lambda row: calculate_air_density(row['temperature_2m'], row['elevation']), axis=1)

In [81]:
fl['elevation'] = fl['elevation'].apply(lambda meter: (meter * 3.28084))

In [82]:
fl['temperature_2m'] = fl['temperature_2m'].apply(lambda celsius: (celsius * 9/5) + 32)

In [83]:
fl['surface_pressure'] = fl['surface_pressure'].apply(lambda hpa: (hpa * 0.02952998057228486))

In [84]:
fl['SpinDirection'] = fl['Tilt'].apply(lambda x: ((int(x.split(':')[0]) * 60 + int(x.split(':')[1])) * 360) / 720)

In [85]:
fl['GyroAngle'] = np.degrees(np.arccos(fl['yt_Efficiency'] / 100))

In [86]:
fl = fl.rename({'SpinAxis': 'SpinDirection'})

In [87]:
fl = fl[(fl['RelSpeed'] >= 25) & (fl['RelSpeed'] <= 120)]
fl = fl[(fl['HorzRelAngle'] >= -10) & (fl['HorzRelAngle'] <= 10)]
fl = fl[(fl['VertRelAngle'] >= -10) & (fl['VertRelAngle'] <= 10)]
fl = fl[(fl['Extension'] >= 0) & (fl['Extension'] <= 10)]
fl = fl[(fl['RelSide'] >= 0) & (fl['RelSide'] <= 6)]
fl = fl[(fl['RelHeight'] >= 0) & (fl['RelHeight'] <= 8)]

fl = fl[(fl['SpinRate'] >= 0) & (fl['SpinRate'] <= 3500)]
fl = fl[(fl['GyroAngle'] >= -90) & (fl['GyroAngle'] <= 90)]
fl = fl[(fl['SpinDirection'] >= 0) & (fl['SpinDirection'] <= 360)]
fl = fl[(fl['yt_SeamLat'] >= 0) & (fl['yt_SeamLat'] <= 180)]
fl = fl[(fl['yt_SeamLong'] >= -180) & (fl['yt_SeamLong'] <= 180)]

In [88]:
fl_pred = fl.rename(columns={
    'temperature_2m': 'Temp_F',
    'relative_humidity_2m': 'RelHumidity',
    'surface_pressure': 'BaroPres_In',
    'elevation': 'Elev_Ft',
    'air_density': 'AirDensity_kgm3',
    'RelSpeed': 'RelSpeed_mph',
    'HorzRelAngle': 'HorzRelAngle_deg',
    'VertRelAngle': 'VertRelAngle_deg',
    'Extension': 'Extension_ft',
    'RelSide': 'RelSide_ft',
    'RelHeight': 'RelHeight_ft',
    'SpinRate': 'SpinRate_rpm',
    'SpinDirection': 'SpinDir_deg',
    'GyroAngle': 'GyroAngle_deg',
    'yt_SeamLat': 'SeamLat_deg',
    'yt_SeamLong': 'SeamLong_deg'
})

fl_pred.insert(0, 'ID', range(1, len(fl_pred) + 1))

In [89]:
fl_pred

,ID,date,Temp_F,RelHumidity,BaroPres_In,Elev_Ft,latitude,longitude,Unnamed: 0,PitchNo,...,RelSpeed_percentile_975,ScaledVelo,SpinRate_percentile_25,SpinRate_percentile_975,ScaledSpin,coord_lat,coord_long,AirDensity_kgm3,SpinDir_deg,GyroAngle_deg
0,1,2023-05-26,74.876901,51.720785,30.012795,1738.84520,46.34,72.54,0,1,...,90.348454,0.888109,1525.865205,2410.273055,0.742418,46.340,72.54,1.153102,45.0,21.883088
1,2,2023-05-26,74.876901,51.720785,30.012795,1738.84520,46.34,72.54,1,2,...,90.348454,0.962236,1525.865205,2410.273055,0.787170,46.340,72.54,1.153102,45.0,21.618729
2,3,2023-05-26,74.876901,51.720785,30.012795,1738.84520,46.34,72.54,2,3,...,90.348454,0.933035,1525.865205,2410.273055,0.819851,46.340,72.54,1.153102,45.0,24.657657
3,4,2023-05-26,74.876901,51.720785,30.012795,1738.84520,46.34,72.54,3,4,...,90.348454,0.844120,1525.865205,2410.273055,0.658319,46.340,72.54,1.153102,52.5,15.243196
4,5,2023-05-26,74.876901,51.720785,30.012795,1738.84520,46.34,72.54,4,5,...,90.348454,0.206006,1525.865205,2410.273055,0.794191,46.340,72.54,1.153102,262.5,83.259765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185835,111203,2023-07-05,90.140903,30.680945,29.546961,3435.03948,40.174,80.24,185440,223,...,90.978419,0.023377,1684.871053,2575.616054,0.928307,40.174,80.24,1.050891,262.5,70.538456
185836,111204,2023-07-05,90.140903,30.680945,29.546961,3435.03948,40.174,80.24,185441,224,...,90.978419,0.160613,1684.871053,2575.616054,0.773394,40.174,80.24,1.050891,270.0,74.944322
185837,111205,2023-07-05,90.140903,30.680945,29.546961,3435.03948,40.174,80.24,185442,225,...,90.978419,1.003742,1684.871053,2575.616054,0.356235,40.174,80.24,1.050891,52.5,16.252663
185838,111206,2023-07-05,90.140903,30.680945,29.546961,3435.03948,40.174,80.24,185443,226,...,90.978419,0.974251,1684.871053,2575.616054,0.263718,40.174,80.24,1.050891,52.5,18.115509


In [90]:
fl_pred_small = fl_pred[fl_pred['Pitcher'] == 'Cole Cook']

In [91]:
fl_pred_small.to_csv('Cole Cook pred.csv')

In [92]:
mov_cook = pd.read_csv('../../Downloads/Cole Cook_11_20_2023, 2_36_02 PM.csv')

In [93]:
cook = pd.merge(fl_pred, mov_cook, on=['ID'])

In [126]:
cook[cook['Seam HB'] == cook['Seam HB'].max()]

,ID,date,Temp_F_x,RelHumidity_x,BaroPres_In_x,Elev_Ft_x,latitude,longitude,Unnamed: 0,PitchNo,...,Magnus IVB,Drag HB,Drag IVB,Seam HB,Seam IVB,Total HB,Total IVB,Apr VA,Apr HA,Zone Time
2,95474,2023-05-12,66.363798,29.019512,29.978836,5679.13404,41.52,88.08,95244,3,...,12.848607,1.49035,-0.226925,2.261302,0.079251,-6.706181,12.700933,-3.233498,-4.105331,0.523558
